# Практикум по социальной инженерии: модули, задачи и решения с пояснениями

# Модуль 1. Введение в социальную инженерию

**Цель:**  Познакомить с основами социальной инженерии, ее определениями и основными понятиями (CIA, DAM, Phishing и др.).

**Пояснения: **Социальная инженерия — это комплекс психологических методов и техник для воздействия на поведение людей с целью получения конфиденциальной информации или доступа к ресурсам. Основные угрозы — фишинг, вишинг, предтекстинг и др.

**Задачи:**

1.Объяснить термины социальной инженерии с помощью словаря Python.

2.Описать основные тактики социальной инженерии и каналы их реализации.

3.Сформировать CSV-файл с распространёнными угрозами и инструментами противодействия.

In [ ]:
TERMS = {
    "phishing": "Метод обмана, направленный на получение конфиденциальной информации через поддельные сообщения.",
    "pretexting": "Создание ложного предлога для получения доверия и информации.",
    "vishing": "Телефонный фишинг, атака через звонки.",
    "baiting": "Использование приманок для получения информации или доступа.",
}

def explain_term(term: str) -> str:
    return TERMS.get(term.lower(), "Термин не найден.")

# Пример использования
print(explain_term("phishing"))



Метод обмана, направленный на получение конфиденциальной информации через поддельные сообщения.


# Модуль 2. OSINT - сбор открытых данных
**Цель:** Научить собирать информацию о цели с помощью общедоступных источников.

**Пояснения:** OSINT (Open Source Intelligence) — технология сбора данных из открытых источников: веб-сайтов, соцсетей, DNS-записей, сертификатов и т.п.

**Задачи:**

Написать скрипт поиска email-адресов по домену.

Определить поддомены с использованием API.

Визуализировать отношения между объектами с помощью графовой библиотеки.

Пример задачи по поддоменам:

In [ ]:
import requests

def find_subdomains(domain):
    url = f"https://api.hackertarget.com/hostsearch/?q={domain}"
    response = requests.get(url)
    return response.text

print(find_subdomains("example.com"))


example.com,23.220.75.245



In [ ]:
import re
import requests

def find_emails(text):
    # Простая регулярка для email
    return re.findall(r'[\w\.-]+@[\w\.-]+', text)

def fetch_emails_from_page(url):
    try:
        response = requests.get(url)
        emails = find_emails(response.text)
        return set(emails)
    except Exception as e:
        print("Ошибка:", e)
        return set()

# Пример
emails = fetch_emails_from_page("https://example.com/contact")
print(emails)


set()


In [ ]:
import requests

def find_subdomains(domain):
    url = f"https://api.hackertarget.com/hostsearch/?q={domain}"
    try:
        response = requests.get(url)
        return response.text.strip()
    except Exception as e:
        return f"Ошибка: {e}"

print(find_subdomains("example.com"))


example.com,23.220.75.245


# Модуль 3. Методы социальной инженерии
**Цель:** Ознакомиться с популярными приемами обмана и их реализацией.

**Пояснения:** Фишинг, вишинг, использование поддельных wifi (Evil Twin), deepfake-технологии — примеры современных атак.

**Задачи:**

Проанализировать шаблоны email для выявления фишинговых признаков.

Реализовать детектор подозрительных ключевых слов в сообщениях.

Написать скрипт выявления поддельных wifi-сетей с анализом SSID и BSSID.

In [ ]:
import re

def is_phishing_email(text):
    suspicious_patterns = [
        r"verify your account",
        r"urgent action required",
        r"click here",
        r"update your password",
    ]
    text_lower = text.lower()
    return any(re.search(pattern, text_lower) for pattern in suspicious_patterns)

email_text = "Please verify your account by clicking here."
print("Phishing detected:", is_phishing_email(email_text))


Phishing detected: True


In [ ]:
suspicious_keywords = ["urgent", "verify", "password", "click here"]

def is_suspicious(text):
    text_lower = text.lower()
    return any(word in text_lower for word in suspicious_keywords)

msg = "Please verify your password by visiting http://example.local/verify"
print("Suspicious message:", is_suspicious(msg))


Suspicious message: True


In [ ]:
def detect_fake_wifi_networks(networks, trusted_ssids):
    # networks — список доступных SSID
    fake_networks = [ssid for ssid in networks if ssid not in trusted_ssids]
    return fake_networks

available_networks = ["HomeWiFi", "Starbucks", "FreeWiFi", "HomeWiFi_5G"]
trusted = ["HomeWiFi", "Starbucks"]
print("Potential fake networks:", detect_fake_wifi_networks(available_networks, trusted))


Potential fake networks: ['FreeWiFi', 'HomeWiFi_5G']


# Модуль 4. Защита и обнаружение атак
**Цель:** Изучить методы защиты и мониторинга.

**Пояснения:** MFA, IDS/IPS, firewall, проверка SSL-сертификатов – основные инструменты в арсенале защиты.

**Задачи:**

Написать парсер логов для выявления подозрительной активности.

Проверить корректность SSL-сертификата сайта.

Проанализировать DMARC/SPF записи домена с помощью скрипта.



In [ ]:
def parse_logs(log_lines):
    suspicious_entries = []
    for line in log_lines:
        if "failed login" in line.lower() or "unauthorized" in line.lower():
            suspicious_entries.append(line)
    return suspicious_entries

logs = [
    "2025-11-19 10:00:00 User admin failed login",
    "2025-11-19 10:05:00 User guest login successful",
    "2025-11-19 10:10:00 Unauthorized access attempt detected",
]
print("Suspicious log entries:", parse_logs(logs))


Suspicious log entries: ['2025-11-19 10:00:00 User admin failed login', '2025-11-19 10:10:00 Unauthorized access attempt detected']


In [ ]:
import ssl
import socket

def check_ssl(hostname):
    try:
        context = ssl.create_default_context()
        with socket.create_connection((hostname, 443), timeout=10) as sock:
            with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                cert = ssock.getpeercert()
                return cert
    except Exception as e:
        return f"SSL error: {e}"

cert_info = check_ssl("google.com")
print("SSL Certificate details:", cert_info)


SSL Certificate details: {'subject': ((('commonName', '*.google.com'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', 'Google Trust Services'),), (('commonName', 'WR2'),)), 'version': 3, 'serialNumber': '9D9ED0E1CC0ECC840A042E3C2F9ED571', 'notBefore': 'Oct 27 08:33:43 2025 GMT', 'notAfter': 'Jan 19 08:33:42 2026 GMT', 'subjectAltName': (('DNS', '*.google.com'), ('DNS', '*.appengine.google.com'), ('DNS', '*.bdn.dev'), ('DNS', '*.origin-test.bdn.dev'), ('DNS', '*.cloud.google.com'), ('DNS', '*.crowdsource.google.com'), ('DNS', '*.datacompute.google.com'), ('DNS', '*.google.ca'), ('DNS', '*.google.cl'), ('DNS', '*.google.co.in'), ('DNS', '*.google.co.jp'), ('DNS', '*.google.co.uk'), ('DNS', '*.google.com.ar'), ('DNS', '*.google.com.au'), ('DNS', '*.google.com.br'), ('DNS', '*.google.com.co'), ('DNS', '*.google.com.mx'), ('DNS', '*.google.com.tr'), ('DNS', '*.google.com.vn'), ('DNS', '*.google.de'), ('DNS', '*.google.es'), ('DNS', '*.google.fr'), ('DNS', '*.google.hu'), ('D

In [ ]:
import dns.resolver

def check_spf(domain):
    try:
        # Set a public DNS server to try and avoid local DNS resolution issues
        resolver = dns.resolver.Resolver()
        resolver.nameservers = ['8.8.8.8'] # Google Public DNS

        answers = resolver.resolve(domain, 'TXT')
        for rdata in answers:
            for txt_string in rdata.strings:
                if b"v=spf1" in txt_string:
                    return txt_string.decode()
    except Exception as e:
        return f"Error: {e}"
    return "SPF record not found."

print(check_spf("google.com"))

v=spf1 include:_spf.google.com ~all


# Модуль 5. Отчетность и стандарты
**Цель:** Освоить подготовку отчетов и стандартизацию процессов.

**Пояснения:** Использование NIST SP 800-53, ISO 27001, MITRE ATT&CK для классификации угроз и формирования рекомендаций.

**Задачи:**

Сформировать JSON-отчёт о выявленных угрозах.

Исследовать техники MITRE ATT&CK по примерам атак.

Подготовить рекомендации по улучшению безопасности на основе отчёта.

In [ ]:
import json
from datetime import datetime

def generate_report(findings, company):
    report = {
        "company": company,
        "date": datetime.now().isoformat(),
        "findings": findings,
        "recommendations": "Рекомендуется обновить ПО и усилить политику безопасности."
    }
    with open("pentest_report.json", "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)
    print("Отчет создан: pentest_report.json")

findings = [
    {"vulnerability": "Open port 22", "risk": "High", "details": "SSH открыт для общего доступа."},
    {"vulnerability": "Expired SSL certificate", "risk": "Medium", "details": "Срок действия сертификата истек."}
]

generate_report(findings, "Example Corp")


Отчет создан: pentest_report.json
